In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gzip
from six.moves import cPickle
import sys    

In [2]:
# selecting parameters
# number of convolutional filters to use
nb_filters = 64 

# filter size    
filter_rows = 1
filter_cols = 2 

# convolution kernel size
nb_conv = 3

# subsampling size 
subsample_rows = 1 
subsample_cols_a = 2    
subsample_cols_b = 1

# batch size 
batch_size = 50

# dimension    
latent_dim = 3
intermediate_dim = 128

# epsilon values 
mean=0.
epsilon_std = 1.0      

# dropout 
drop = 0

# epoch, iteration sizes  
nb_epoch = 1  

# saving models: start & end
nb_start = 0
nb_end = 400       
total_epochs = (nb_end-nb_start)*nb_epoch  
print "Total of %i epochs are set to run" % total_epochs  

Total of 400 epochs are set to run


In [3]:
import os, sys
path_data = './X_110000.pkl.gz' 
path_1 = "./fig"
path_2 = "./imgs"
path_3 = "./hist"
path_4 = "./model"
if not os.path.exists(path_1):
    os.mkdir(path_1, 0755);
if not os.path.exists(path_2):
    os.mkdir(path_2, 0755);
if not os.path.exists(path_3):
    os.mkdir(path_3, 0755);
if not os.path.exists(path_4):
    os.mkdir(path_4, 0755);   
print "directories creation compledted or if already exist - then checked"   

directories creation compledted or if already exist - then checked


In [4]:
with gzip.open(path_data, 'rb') as f3:    
    (x_train_raw, y_train_raw), (x_test_raw,y_test_raw), (x_pred_raw,y_pred_raw) = cPickle.load(f3)  


In [5]:
# normalizing input image matrix 
a = np.amax(x_train_raw)
b = np.amax(x_test_raw)
c = np.amax(x_pred_raw)
x_train = x_train_raw.astype('float32') / a
x_test = x_test_raw.astype('float32') / b
x_pred = x_pred_raw.astype('float32') / c
print x_train.shape

(60000, 784)


In [6]:
imag_rows = x_train_raw.shape[0]/len(x_train_raw)
imag_cols = x_train_raw.shape[1]
imag_chns = 1
print imag_rows, imag_cols

1 784


In [8]:
from neon.layers.layer import Layer, interpret_in_shape
class Lambda(Layer):
    '''
    Take the input from previous layer and calculate the latent variable
    '''

    # constructor and initialize buffers
    def __init__(self, nout, name=None):
        super(Lambda, self).__init__(name)
        self.nout = nout
        
        # required attributes
        self.inputs = None  #..?
        self.in_shape = None  # shape of the inputs to this layer
        self.out_shape = None  # shape of the outputs from this layer

    # configure the layer input and output shapes
    def configure(self, in_obj):
        super(Lambda, self).configure(in_obj)
        (self.nin, self.nsteps) = interpret_in_shape(self.in_shape)
        self.out_shape = (self.nout, self.nsteps)
        return self
        return self

    # compute the fprop
    def fprop(self, inputs, inference=False):
        self.inputs = inputs
        
        def sampling(data):
            z_mean = z_log_var = data
            epsilon = np.random.normal(0, 1.0, (data.shape))
            return z_mean + np.exp(z_log_var)*epsilon
        
        for c in range(self.outputs.shape[1]):
            self.outputs[:,c] = sampling(self.inputs[:,c])
        return self.outputs

    # backprop the gradients
    def bprop(self, error):
        if self.deltas is None:
            self.deltas = error.reshape(self.in_shape, self.be.bsz)
        self.deltas = error
        return self.deltas

In [10]:
import logging
from neon.callbacks.callbacks import Callbacks
from neon.data import ArrayIterator
from neon.initializers import Gaussian
from neon.layers import GeneralizedCost, Affine, Reshape
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Rectlin, Logistic, CrossEntropyBinary, Tanh
from neon.transforms import Misclassification, Accuracy, Softmax
from neon.layers import Conv, Affine, Pooling, Deconv, Dropout
from neon.initializers import GlorotUniform

In [11]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu', batch_size=1)

In [13]:
train = ArrayIterator(X=x_train, lshape=(1, 28, 28))
train_2 = ArrayIterator(x_train)

In [14]:
init = GlorotUniform()
activation = Rectlin()

In [1]:
# Construct Variance-Autoencoder layers.

# layers = [Conv(fshape=(1, 1, 2), init=init, activation=activation, padding={"pad_h": 1}),
#          Conv(fshape=(64, 1, 2), init=init, strides={"str_w":1, "str_h":2},
#               activation=activation, padding={"pad_h":1}),
#          Conv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
#               activation=activation, padding={"pad_h":2, "pad_w":2}),
#          Conv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
#               activation=activation, padding={"pad_h":2, "pad_w":2}),
#          Reshape(64*28*28),
#           Affine(nout=128, init=init, activation=activation),
#           Affine(nout=3, init=init, activation=activation),
#          Lambda(),
#           Affine(nout=128, init=init, activation=activation),
#           Affine(nout=64*28*28, init=init, activation=activation),
#           Reshape((64, 28, 28)),
#           Deconv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
#                 activation=activation, padding={"pad_h":2, "pad_w":2}),
#           Deconv(fshape=(64, 3, 3), init=init, strides={"str_w":1, "str_h":1},
#                 activation=activation, padding={"pad_h":2, "pad_w":2}),
#           Deconv(fshape=(64, 1, 2), init=init, strides={"str_w":1, "str_h":2},
#                 activation=activation, padding={"pad_h":1}),
#           Deconv(fshape=(1, 1, 2), init=init, strides={"str_w":1, "str_h":2},
#                 activation=Tanh(), padding={"pad_h":1})
#          ]

Test reshape and Lambda layer

In [16]:
layers = [Affine(nout=128, init=init, activation=activation),
          Reshape(reshape=(2,64)),
          Reshape(reshape=128),
          Affine(nout=3, init=init, activation=activation),
          Lambda(nout=3),
          Affine(nout=128, init=init, activation=activation),
          Affine(nout=28*28, init=init, activation=activation)]

In [17]:
model = Model(layers=layers)
cost = GeneralizedCost(costfunc=CrossEntropyBinary())
rms = RMSProp(learning_rate=0.001, epsilon=1e-08)
callbacks = Callbacks(model, eval_set=train, eval_freq=5)

In [ ]:
model.fit(train_2, optimizer=rms, num_epochs=5, cost=cost, callbacks=callbacks)